In [340]:
!pip3 install plotly
!pip3 install pandas-profiling

In [11]:
from bs4 import BeautifulSoup as soup
from datetime import date, datetime
from urllib.request import Request, urlopen


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [254]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import gc
import warnings
warnings.filterwarnings("ignore")

from pandas_profiling import ProfileReport

# Extracting Data - Web Scrapping

In [227]:
url = "https://www.worldometers.info/coronavirus/#countries"

In [228]:
req = Request(url, headers={"User-Agent":"Mozilla/5.0"})

In [229]:
webpage = urlopen(req)
# data_ext = page_soup
data_ext = soup(webpage, "html.parser")

In [230]:
today = datetime.now()

In [231]:
yesterday = "%s %d,%d" %(date.today().strftime("%b"), today.day-1, today.year)
yesterday

'Jan 14,2022'

In [232]:
table = data_ext.findAll("table", {"id":"main_table_countries_yesterday"})

In [233]:
containers = table[0].findAll("tr",{"style":""})

In [234]:
title = containers[0]
del containers[0]

clean = True
data = []

for country in containers:
    country_data = []
    country_cont = country.findAll("td")
    
    if country_cont[1].text == "China":
        continue
    
    for i in range(1, len(country_cont)):
        final_feat = country_cont[i].text
        if clean:
            if i != 1 and i != len(country_cont)-1:
                final_feat = final_feat.replace(",","")
                
                if final_feat.find('+') != -1:
                    final_feat = final_feat.replace('+','')
                    final_feat = float(final_feat)
                elif final_feat.find('-') != -1:
                    final_feat = final_feat.replace('-','')
                    final_feat = float(final_feat)*-1
        
        if final_feat == 'N/A':
            final_feat = 0
        elif final_feat == "" or final_feat == " ":
            final_feat = -1
            
        
        country_data.append(final_feat)
        
    data.append(country_data)

In [235]:
data

[['World',
  '324183092',
  3234667.0,
  '5547481',
  7883.0,
  '265315520',
  1192025.0,
  '53320091',
  '96265',
  '41590',
  '711.7',
  -1,
  -1,
  -1,
  'All',
  '\n',
  -1,
  -1,
  -1,
  -1,
  -1],
 ['USA',
  '66209535',
  827132.0,
  '872086 ',
  2303.0,
  '43019924',
  111883.0,
  '22317525',
  '25636',
  '198243',
  '2611',
  '856423446',
  '2564277',
  '333982423 ',
  'North America',
  '5',
  '383',
  '0',
  '2477',
  '7',
  '66,822'],
 ['India',
  '36850962',
  268833.0,
  '485780 ',
  430.0,
  '34947390',
  122684.0,
  '1417792',
  '8944',
  '26307',
  '347',
  '699099084',
  '499061',
  '1400828197 ',
  'Asia',
  '38',
  '2884',
  '2',
  '192',
  '0.3',
  '1,012'],
 ['Brazil',
  '22927203',
  111376.0,
  '620847 ',
  238.0,
  '21710831',
  -1,
  '595525',
  '8318',
  '106698',
  '2889',
  '63776166',
  '296801',
  '214878863 ',
  'South America',
  '9',
  '346',
  '3',
  '518',
  '1',
  '2,771'],
 ['UK',
  '15065407',
  99322.0,
  '151612 ',
  270.0,
  '11208701',
  81748.

In [236]:
df = pd.DataFrame(data)

In [237]:
df.drop([15, 16, 17, 18, 19, 20], axis=1, inplace=True)

In [238]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,World,324183092,3234667.0,5547481,7883.0,265315520,1192025.0,53320091,96265,41590,711.7,-1,-1,-1,All
1,USA,66209535,827132.0,872086,2303.0,43019924,111883.0,22317525,25636,198243,2611,856423446,2564277,333982423,North America
2,India,36850962,268833.0,485780,430.0,34947390,122684.0,1417792,8944,26307,347,699099084,499061,1400828197,Asia
3,Brazil,22927203,111376.0,620847,238.0,21710831,-1.0,595525,8318,106698,2889,63776166,296801,214878863,South America
4,UK,15065407,99322.0,151612,270.0,11208701,81748.0,3705094,777,220147,2215,430233640,6286888,68433484,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Palau,56,16.0,-1,-1.0,14,-1.0,42,-1,3073,-1,19159,1051191,18226,Australia/Oceania
213,Solomon Islands,25,-1.0,-1,-1.0,20,-1.0,5,-1,35,-1,4500,6314,712704,Australia/Oceania
214,Western Sahara,10,-1.0,1,-1.0,8,-1.0,1,-1,16,2,-1,-1,619851,Africa
215,Marshall Islands,7,-1.0,-1,-1.0,4,-1.0,3,-1,117,-1,-1,-1,59803,Australia/Oceania


In [239]:
labels = ['Country', 'Total Cases', 'New Cases', 'Total Deaths', 'New Deaths', 'Total Recovered', 'New Recovered',
         'Active Cases', 'Serious Critical', 'Tot Cases/1M pop', 'Deaths/1M', 'Total Tests', 'Tests/1M', 
          'Population', 'Continent']

# 'New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop'

In [240]:
df.columns = labels

In [241]:
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious Critical,Tot Cases/1M pop,Deaths/1M,Total Tests,Tests/1M,Population,Continent
0,World,324183092,3234667.0,5547481,7883.0,265315520,1192025.0,53320091,96265,41590,711.7,-1,-1,-1,All
1,USA,66209535,827132.0,872086,2303.0,43019924,111883.0,22317525,25636,198243,2611,856423446,2564277,333982423,North America
2,India,36850962,268833.0,485780,430.0,34947390,122684.0,1417792,8944,26307,347,699099084,499061,1400828197,Asia
3,Brazil,22927203,111376.0,620847,238.0,21710831,-1.0,595525,8318,106698,2889,63776166,296801,214878863,South America
4,UK,15065407,99322.0,151612,270.0,11208701,81748.0,3705094,777,220147,2215,430233640,6286888,68433484,Europe


In [151]:
# type(df['Active Cases/1M pop'][0])

# """
# function to remove commas from the Active Cases/1M pop

# Not Successful
# """
# def remove_comma(a):
#     print('inside func')
#     for x in a:
#         print(x)
#         print(x.replace(",",""))

int

In [242]:
# df.drop(['New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop'], axis=1, inplace=True)

# df.head()

In [243]:
for label in df.columns:
    if label != 'Country' and label != 'Continent':
        df[label] = pd.to_numeric(df[label])

In [225]:
"""
Create 3 new columns
1. percentage increase in cases
2. percentage increase in deaths
3. percentage increase in recovered
"""

In [244]:
df['%inc Cases'] = df['New Cases']/df['Total Cases']*100
df['%inc Deaths'] = df['New Deaths']/df['Total Deaths']*100
df['%inc Recovered'] = df['New Recovered']/df['Total Recovered']*100

In [247]:
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious Critical,Tot Cases/1M pop,Deaths/1M,Total Tests,Tests/1M,Population,Continent,%inc Cases,%inc Deaths,%inc Recovered
0,World,324183092,3234667.0,5547481,7883.0,265315520,1192025.0,53320091,96265,41590.0,711.7,-1,-1,-1,All,0.997790,0.142101,0.449286
1,USA,66209535,827132.0,872086,2303.0,43019924,111883.0,22317525,25636,198243.0,2611.0,856423446,2564277,333982423,North America,1.249264,0.264079,0.260073
2,India,36850962,268833.0,485780,430.0,34947390,122684.0,1417792,8944,26307.0,347.0,699099084,499061,1400828197,Asia,0.729514,0.088517,0.351053
3,Brazil,22927203,111376.0,620847,238.0,21710831,-1.0,595525,8318,106698.0,2889.0,63776166,296801,214878863,South America,0.485781,0.038335,-0.000005
4,UK,15065407,99322.0,151612,270.0,11208701,81748.0,3705094,777,220147.0,2215.0,430233640,6286888,68433484,Europe,0.659272,0.178086,0.729326


# Exploratory Data Analysis

In [275]:
cases = df[['Total Recovered', 'Active Cases', 'Total Deaths']].loc[0]

cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ['Type', 'Total']

cases_df['Percentage'] = np.round(100*cases_df['Total']/np.sum(cases_df['Total']), 2)
cases_df['Virus'] = ['Covid-19' for i in range(len(cases_df))]

In [276]:
cases_df

,Type,Total,Percentage,Virus
0,Total Recovered,265315520,81.84,Covid-19
1,Active Cases,53320091,16.45,Covid-19
2,Total Deaths,5547481,1.71,Covid-19


In [288]:
fig = px.bar(cases_df, x='Virus', y='Percentage', color='Type', hover_data=['Total'], title='World COVID Cases Analysis')
fig.show()

### Of the total cases, we can see above that 81.84% have recovered from the Covid-19 virus, 16.45% are total Active Cases and 1.71%a re the total Deaths

### Since the recovery rate is very high, even if the COVID-19 cases right now are sky rocketing, there is not much threat to life as it was during the start of 2020. We can also observe that the death rate is only 1.71%. From this we can safely conclude that the reason behind such high recovery rate is that many people have been vaccinated, they have taken the 2 shots, some have also taken the booster shot. As to examine the death cases, one should dig more into getting data of the people who died, were vaccinated or not not vaccinated?

### Above we calculated for the world, now lets calculate the same for USA and INDIA

In [289]:
cases_usa = df[['Total Recovered', 'Active Cases', 'Total Deaths']].loc[1]

casesUS_df = pd.DataFrame(cases_usa).reset_index()
casesUS_df.columns = ['Type', 'Total']

casesUS_df['Percentage'] = np.round(100*casesUS_df['Total']/np.sum(casesUS_df['Total']), 2)
casesUS_df['Virus'] = ['Covid-19' for i in range(len(casesUS_df))]

In [290]:
casesUS_df

,Type,Total,Percentage,Virus
0,Total Recovered,43019924,64.98,Covid-19
1,Active Cases,22317525,33.71,Covid-19
2,Total Deaths,872086,1.32,Covid-19


In [291]:
fig = px.bar(casesUS_df, x='Virus', y='Percentage', color='Type', hover_data=['Total'], title='USA COVID Cases Analysis')
fig.show()

#### From the above graph we can see that 64.98% have recovered in USA from COVID-19, 33.71% are active cases and 1.32% is the percentage of total deaths. This is not bad and also not good at the same time, as we would want the total recovery more and total active cases less

In [302]:
"""
We will plot a similar graph for New cases, deaths and recoveries
"""

cases = df[['New Cases', 'New Recovered', 'New Deaths']].loc[0]

cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ['Type', 'Total']

cases_df['Percentage'] = np.round(100*cases_df['Total']/np.sum(cases_df['Total']), 2)
cases_df['Virus'] = ['Covid-19' for i in range(len(cases_df))]

In [303]:
fig = px.bar(percent_df, x='Virus', y='Percentage', color = 'Type', hover_data=['Total'], title='USA New COVID Cases Analysis')
fig.show()

In [309]:
"""
We will plot a similar graph for New cases, deaths and recoveries
"""

percent = np.round(df[['%inc Cases', '%inc Deaths', '%inc Recovered']].loc[0], 2)

percent_df = pd.DataFrame(percent).reset_index()
percent_df.columns = ['Type','Percentage']

fig = go.Figure()

fig.add_trace(go.Bar(x = percent_df['Type'], y = percent_df['Percentage']))
fig.show()

## Visualization of attributes Continent wise

In [314]:
continent = df.groupby('Continent').sum().drop('All').reset_index()
continent

,Continent,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious Critical,Tot Cases/1M pop,Deaths/1M,Total Tests,Tests/1M,Population,%inc Cases,%inc Deaths,%inc Recovered
0,Africa,10425889,42177.0,233744,290.0,9217908,39752.0,937038,2531,1317751.0,16224.0,90364948,9253488,1389794811,26.045310,-104.467870,14.695669
1,Asia,89176221,524112.0,1264795,1086.0,83795950,257689.0,4115475,25910,2720006.0,26440.0,1589594505,68488040,3222107916,20.455607,67.892834,7.351464
2,Australia/Oceania,1708244,135288.0,4868,49.0,573788,566.0,1083212,391,388820.0,4820.0,64935415,7231742,42444178,21.503699,285.479420,-35.668993
3,Europe,102455990,1223120.0,1568358,3142.0,80515876,569122.0,20371756,21362,7989534.0,96048.0,1819502187,154796352,748322555,64.287333,-3.562140,49.411847
4,North America,77661159,936335.0,1273626,2634.0,52439674,184569.0,23939724,31802,4315472.0,42933.0,959474988,65648315,596194759,67.645710,-97.429195,33.436953
5,South America,42650247,373399.0,1197432,596.0,36467587,139569.0,2106919,14199,1258800.0,28421.0,192458538,10776890,436215165,16.478797,102.301668,5.415211


In [335]:
def visualizaing_continent(list_arg):
    for x in list_arg:
        cont_df = continent[['Continent', x]]
        cont_df['Percentage'] = np.round(100*cont_df[x]/np.sum(cont_df[x]), 2)
        cont_df['Virus'] = ['COVID-19' for i in range(len(cont_df))]
    
        fig = px.bar(cont_df, x='Virus', y='Percentage', color = 'Continent', hover_data=[x])
        fig.show()
        gc.collect()

In [336]:
cases = ["Total Cases", "Active Cases", "Serious Critical", "Tot Cases/1M pop"]

deaths = ["Total Deaths", "New Deaths", "Deaths/1M"]

recovered = ["Total Recovered", "New Recovered", "%inc Recovered"]

### Visualization for Cases

In [337]:
visualizaing_continent(cases)

## Visualization for Deaths

In [338]:
visualizaing_continent(deaths)

## Visualization for Recovered

In [339]:
visualizaing_continent(recovered)

### Plotting with respect to countries for selected attributes.

In [350]:
country_df = df.drop([len(df)-1])
country_df.drop([0], inplace=True)
country_df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious Critical,Tot Cases/1M pop,Deaths/1M,Total Tests,Tests/1M,Population,Continent,%inc Cases,%inc Deaths,%inc Recovered
1,USA,66209535,827132.0,872086,2303.0,43019924,111883.0,22317525,25636,198243.0,2611.0,856423446,2564277,333982423,North America,1.249264,0.264079,0.260073
2,India,36850962,268833.0,485780,430.0,34947390,122684.0,1417792,8944,26307.0,347.0,699099084,499061,1400828197,Asia,0.729514,0.088517,0.351053
3,Brazil,22927203,111376.0,620847,238.0,21710831,-1.0,595525,8318,106698.0,2889.0,63776166,296801,214878863,South America,0.485781,0.038335,-0.000005
4,UK,15065407,99322.0,151612,270.0,11208701,81748.0,3705094,777,220147.0,2215.0,430233640,6286888,68433484,Europe,0.659272,0.178086,0.729326
5,France,13569675,329371.0,126721,191.0,8961841,104386.0,4481113,3939,207186.0,1935.0,209869764,3204349,65495283,Europe,2.427258,0.150725,1.164783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Macao,79,-1.0,-1,-1.0,77,-1.0,2,-1,119.0,-1.0,5075,7656,662917,Asia,-1.265823,100.000000,-1.298701
212,Palau,56,16.0,-1,-1.0,14,-1.0,42,-1,3073.0,-1.0,19159,1051191,18226,Australia/Oceania,28.571429,100.000000,-7.142857
213,Solomon Islands,25,-1.0,-1,-1.0,20,-1.0,5,-1,35.0,-1.0,4500,6314,712704,Australia/Oceania,-4.000000,100.000000,-5.000000
214,Western Sahara,10,-1.0,1,-1.0,8,-1.0,1,-1,16.0,2.0,-1,-1,619851,Africa,-10.000000,-100.000000,-12.500000


In [364]:
def plot_top_countries(data, n):
    """
    logic: top n countries to be plotted for all the required parameters passed
    Build the for loop for iterating through the index
    once iterated through the index, use this index to plot the passed required attributes on the graph
    
    You will also need a counter which will increase with each loop until it becomes greater than the passed value 
    of n.
    """
    
    fig = go.Figure()
    c = 0
    
    for i in country_df.index:
        if c < n:
            fig.add_trace(go.Bar(name=country_df['Country'][i], x = data, y=country_df.loc[i][1:14]))
        else:
            break
        
        c = c+1;
        
    fig.update_layout(title = {'text':f'top {n} countries for Covid analysis'}, yaxis_type='log') 
    fig.show()
    
# Used the yaxis_type parameter as log to normalize the data for better graph

In [365]:
plot_top_countries(country_df.columns[1:14], 5)

In [377]:
# Plotting top 10 countries for selected parameters

selected_att = ['Total Cases', 'Total Deaths', 'Total Recovered', 'Active Cases', 'Serious Critical',
                'Tot Cases/1M pop', 'Deaths/1M', 'Total Tests', 'Tests/1M','Population']

plot_top_countries(temp, 10)